# Query BIDS dataset and extract full paths to data files

This script uses BIDSLayout to query BIDS dataset and extract:
- anatomical images: `anat_files` 
- brain masks: `mask_files`
- confounds: `conf_files`
- functional images: `fmri_files`

All paths are stored as JSON file. Anatomical images paths are stored as a list of files (`anat_filenames.json`), whereas brain masks, confounds and functional images are stored as a dict with two keys corresponding to task condtions (`prlrew` and `prlpun`) and values being list of files for all subjects.

In [1]:
import json
from os.path import join

from bids import BIDSLayout
from dn_utils.path import path

In [2]:
layout = BIDSLayout(
    root=path["bids"],
    derivatives=True,
    validate=True,
    index_metadata=False
)

anat_filter = {
    "extension": [".nii.gz"],
    "space": "MNI152NLin2009cAsym",
    "suffix": "T1w",
    "desc": "preproc",
    "return_type": "filename"
}

fmri_filter = {
    "extension": [".nii", ".nii.gz"],
    "space": "MNI152NLin2009cAsym",
    "suffix": "bold",
    "desc": "preproc",
    "return_type": "filename"
}

conf_filter = {
    "extension": "tsv",
    "desc": "confounds",
    "return_type": "filename"
}

mask_filter = {
    "extension": [".nii.gz"],
    "space": "MNI152NLin2009cAsym",
    "desc": "brain",
    "suffix": "mask",
    "return_type": "filename"
}

anat_files = layout.get(**anat_filter)

fmri_files, conf_files, mask_files = {}, {}, {}

for task_dict in [{"task": "prlrew"}, {"task": "prlpun"}]:
    fmri_filter.update(task_dict)
    conf_filter.update(task_dict)
    mask_filter.update(task_dict)
    fmri_files[task_dict["task"]] = layout.get(**fmri_filter)
    conf_files[task_dict["task"]] = layout.get(**conf_filter)
    mask_files[task_dict["task"]] = layout.get(**mask_filter)

In [3]:
variable_file_map = {
    "fmri_filenames.json": fmri_files,
    "conf_filenames.json": conf_files,
    "mask_filenames.json": mask_files,
    "anat_filenames.json": anat_files,
}

for filename, variable in variable_file_map.items():
    with open(join(path["data_paths"], filename), "w") as f:
        f.write(json.dumps(variable, indent=4))